In [2]:
# Importing the required libraries
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
import pandas as pd
import pytorch_lightning as pl

# Setting the random seed for reproducibility
random_seed = 42
torch.manual_seed(random_seed)

# Defining the batch size, available GPUs, and number of workers
BATCH_SIZE=1
AVAIL_GPUS = min(1, torch.cuda.device_count())
NUM_WORKERS=int(os.cpu_count() / 2)

c:\Users\SMEET_SIMUL\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This block of code imports various libraries required for building a deep learning model using PyTorch. It also sets the random seed to 42 for reproducibility of results.

Furthermore, it defines the batch size for the data loader, the number of available GPUs, and the number of workers for the data loader to use.
***

In [3]:
# Define CustomDataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.FFT_files = os.listdir(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.FFT_files)

    def __getitem__(self, idx):
        file_name=self.FFT_files[idx]
        FFT_path = os.path.join(self.root_dir, file_name)
        FFT = pd.read_csv(FFT_path,header=None).values
        FFT_tensor = torch.tensor(FFT, dtype=torch.float32)
        FFT_tensor = FFT_tensor.view(-1, FFT_tensor.shape[1])
        Label_Tag=file_name.split('_')
        label=[]
        for n in [1,3,5,7,9,11,13]:
            temp_num=float(Label_Tag[n])
            label.append(temp_num)
        label=torch.tensor(label)
    
        return FFT_tensor, label

In [5]:
## Converting the 2D data matrix to a 3D matrix

class CustomDataModule(pl.LightningDataModule):
    def __init__(self, csv_root, transform=None, batch_size=32, num_workers=0):
        super().__init__()
        self.csv_root = csv_root   # Directory path where the CSV file is stored
        self.batch_size = batch_size   # The batch size for the data loader
        self.num_workers = num_workers   # The number of worker processes for loading the data
        self.transform = transform   # Optional data transformation to be applied

    def prepare_data(self):
        pass   # Placeholder for any data preparation step, if needed

    def setup(self, stage=None):
        self.dataset = CustomDataset(self.csv_root, transform=self.transform)   # Initialize the CustomDataset class with the specified CSV directory and transform
        #self.dataset_train, self.dataset_val = random_split(self.dataset,[int(len(self.dataset)*0.7),len(self.dataset)-int(len(self.dataset)*0.7)])
        self.dataset_train = self.dataset   # Set the training dataset to be the entire dataset

    def train_dataloader(self):
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)   # Return a DataLoader object for the training data, which shuffles the data and divides it into batches

    def val_dataloader(self):
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)   # Return a DataLoader object for the validation data, which is set to be the same as the training data

    def test_dataloader(self):
        return None   # No test data is used for this model, so return None


In [6]:
# Setting the root directory of the dataset
root_dir = '../Data/FFT_Data/Turn_off'

# Defining the transformations for the dataset using PyTorch's Compose function
# transformations = transforms.Compose([
#     transforms.ToTensor(),
# ])

# Creating a custom dataset and dataloader using the CustomDataModule class
data_module = CustomDataModule(root_dir, transform=None)

In [7]:
# Set up the data module and data loader
data_module.setup()

# Retrieve the training data from the data loader
train_dataloader = data_module.train_dataloader()

# Get the first batch of data from the data loader
i, l = next(iter(train_dataloader))

In [8]:
# Printing the shape of the input data and labels
print(i.shape)
print(l.shape)

torch.Size([32, 3, 981])
torch.Size([32, 7])


### Discriminator



In [9]:
# A discriminator model that determines if an image is real or fake, outputting a single value between 0 and 1

## TODO: Change the Channel and input size

# Start [batch, 3, 981] using 1-D Convolution, and using dialation for using data diffenrent part data 
# [batch, 3, 981] > [batch, 3, 979] > [batch, 3, 975] > [batch, 3, 967] > [batch, 3, 951] > [batch, 3, 919] > [batch, 3, 855] > [batch, 3, 727] > [batch, 3, 471] <[batch, 1, 469] 
# linear [469, 200, 1]
#  
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        # CNN architecture with dialation  references by TCN
        self.conv1 = nn.Conv1d(3, 3, kernel_size=3,dilation=1)
        self.conv2 = nn.Conv1d(3, 3, kernel_size=3,dilation=2)
        self.conv3 = nn.Conv1d(3, 3, kernel_size=3,dilation=4)
        self.conv4 = nn.Conv1d(3, 3, kernel_size=3,dilation=8)
        self.conv5 = nn.Conv1d(3, 3, kernel_size=3,dilation=16)
        self.conv6 = nn.Conv1d(3, 3, kernel_size=3,dilation=32)
        self.conv7 = nn.Conv1d(3, 3, kernel_size=3,dilation=64)
        self.conv8 = nn.Conv1d(3, 3, kernel_size=3,dilation=128)
        self.conv9 = nn.Conv1d(3, 1, kernel_size=3)

        # FCN
        self.fc1 = nn.Linear(469, 200)
        self.fc2 = nn.Linear(200, 1)

        
  
    def forward(self, x):
        # Apply convolutional and ReLU activation
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        # Flatten the tensor so it can be fed into the fully connected layers
        temp_length=x.shape[2]
        x = x.view(-1, temp_length)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)

        # Apply sigmoid activation to output a value between 0 and 1
        return torch.sigmoid(x)

### Generator

In [ ]:
## A generator model that takes a latent space vector as input and outputs a wavelet image

## TODO: Change the Channel and input size
# Input_size [batch,7]
# End_size [batch, 3, 981]
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        # Define the layers of the generator
        self.lin1 = nn.Linear(7, 7*7*64)  # [n, 64, 7, 7]
        self.ct1 = nn.ConvTranspose2d(64, 32, 4, stride=2) # [n, 32, 16, 16]
        self.ct2 = nn.ConvTranspose2d(32, 16, 4, stride=2) # [n, 16, 34, 34]
        self.conv = nn.Conv2d(16, 1, kernel_size=7)  # [n, 1, 28, 28]
    

    def forward(self, x):
        # Pass the input through a linear layer and reshape
        x = self.lin1(x)
        x = F.relu(x)
        x = x.view(-1, 64, 7, 7)  #256
        
        # Upsample to 16x16 (64 feature maps)
        x = self.ct1(x)
        x = F.relu(x)
        
        # Upsample to 34x34 (16 feature maps)
        x = self.ct2(x)
        x = F.relu(x)
        
        # Apply a convolutional layer to produce a 28x28 wavelet image with a single channel
        return self.conv(x)

This block of code defines a PyTorch module called ```Generator``` that takes a latent space vector as input and generates a wavelet image as output.

The ```nn.Module``` class is used as the parent class for this custom module.

The ```__init__``` method initializes the layers of the generator. The linear layer is used to transform the input latent space vector to a 3D tensor. The convolutional transpose layers are used to upsample the tensor, increasing the spatial dimensions and decreasing the number of channels. The final convolutional layer produces a 28x28 wavelet image with a single channel.

The ```forward``` method takes the input tensor ```x``` and passes it through the layers of the generator, applying ReLU activation after each layer. The final output is a wavelet image of size 28x28 with a single channel.

The ```TODO``` comment indicates that the channel and input size should be changed to fit the specific dataset being used.
***

### GAN

In [ ]:
## TODO: Change the input Data using the label

class GAN(pl.LightningDataModule):
    def __init__(self, latent_dim=100, lr=0.0002):
        super().__init__()
        # Save the hyperparameters and initialize the generator and discriminator
        self.save_hyperparameters()
        self.generator = Generator(latent_dim=self.hparams.latent_dim)
        self.discriminator = Discriminator()
        
        # Create validation noise vector
        self.validation_z = torch.randn(6, self.hparams.latent_dim)
        
    def forward(self, z):
        return self.generator(z)
    
    def adverarial_loss(self, y_hat,y):
        # Calculate binary cross-entropy loss
        return F.binary_cross_entropy(y_hat, y)
    
    def training_step(self, batch, batch_dim, optimizer_idx):   
        real_imgs, labels = batch
        
        # Sample noise
        z = torch.randn(real_imgs.shape[0], self.hparams.latent_dim)
        z = z.type_as(real_imgs)
        
        if optimizer_idx == 0:
            # Train the generator: maximize log(D(G(z)))
            fake_imgs = self(z)
            y_hat = self.discriminator(fake_imgs)
            
            y = torch.ones(real_imgs.size(0), 1)
            y = y.type_as(real_imgs)
            
            g_loss = self.adverarial_loss(y_hat, y)
            
            log_dict = {"g_loss" : g_loss }
            return {"loss": g_loss, "progress bar" : log_dict, "log": log_dict}
        
        if optimizer_idx == 1:
            # Train the discriminator: maximize log(D(x)) + log(1 - D(G(z)))
            y_hat_real = self.discriminator(real_imgs)
            y_real = torch.ones(real_imgs.size(0), 1)
            y_real = y_real.type_as(real_imgs)
            real_loss = self.adverarial_loss(y_hat_real, y_real)
            
            y_hat_fake = self.discriminator(self(z))
            y_fake = torch.zeros(real_imgs.size(0), 1)
            y_fake = y_fake.type_as(real_imgs)
            fake_loss = self.adverarial_loss(y_hat_fake, y_fake)
            
            d_loss = (real_loss + fake_loss) / 2
            
            log_dict = {"d_loss" : d_loss }
            return {"loss": d_loss, "progress bar" : log_dict, "log": log_dict}
                
    def configure_optimizers(self):
        lr=self.hparams.lr
        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr)
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr)
        return [opt_g, opt_d], []
    
    
    def on_epoch_end(self):
        # Call plot_imgs() at the end of each epoch
        self.plot_imgs()

This code block defines a PyTorch Lightning module called ```GAN``` that implements a generative adversarial network (GAN).

The ```__init__``` method initializes the generator and discriminator networks, and creates a noise vector for validation.

The ```forward``` method of the ```GAN``` class passes the noise vector through the generator network to generate images.

The ```adverarial_loss``` method calculates the binary cross-entropy loss.

The ```training_step``` method trains the generator and discriminator using the binary cross-entropy loss. It first trains the generator and then the discriminator.

The ```configure_optimizers``` method sets the optimizers for the generator and discriminator.

The ```plot_imgs``` method generates validation images and plots them.

The ```on_epoch_end``` method is called at the end of each epoch and it calls the ```plot_imgs``` method.
***

In [ ]:
# Set up the MNIST data module


# Create an instance of the GAN model
model = GAN()

This code block creates an instance of the ```MNISTDataModule``` class and assigns it to the variable ```dm```. It also creates an instance of the ```GAN``` class and assigns it to the variable ```model```.
***

In [ ]:
model.plot_imgs()

```model.plot_imgs()``` generates validation images using the generator network and plots them. This method is defined in the ```GAN``` class and uses the validation noise vector ```validation_z``` defined in the constructor to generate the images. It then plots the generated images using Matplotlib.
***

In [ ]:
# Set up the trainer object
trainer = pl.Trainer(max_epochs=20, gpus=AVAIL_GPUS)

# Train the model
trainer.fit(model)

This code block sets up a ```Trainer``` object with a maximum number of epochs and the number of available GPUs. It then trains the ```model``` using the ```Trainer``` object and the ```dm``` data module.

The ```fit``` method of the ```Trainer``` object trains the ```model``` for the specified number of epochs using the data provided by the data module. During training, the ```on_epoch_end``` method of the ```GAN``` model is called after each epoch, which generates and plots validation images.

After training is complete, the ```Trainer``` object returns a ```TrainerResult``` object with information about the training process, such as the final training and validation losses.
***